In [ ]:
#This cell will attempt to read the directory file using pandas - I like the output! 
import pandas
import re
import phonenumbers
df = pandas.read_csv('InstantChurchDirectory-20210218-042000.csv')

# Set missing values to empty strings
df['Child_Names'] = df['Child_Names'].fillna('')
df['Address'] = df['Address'].fillna('')  
df['City'] = df['City'].fillna('')
df['State'] = df['State'].fillna('')
df['Phone_Numbers'] = df['Phone_Numbers'].fillna('')
df['Emails'] = df['Emails'].fillna('')
df['Additional_Details'] = df['Additional_Details'].fillna('')
df['Notes'] = df['Notes'].fillna('')

# Print a sample
df.head(5)

In [ ]:
# Not Needed: This cell will attempt to add a column that concatenates all names

def concat_names(row):
    # this function will concatenate the names from  pandas dataframe columns
    # Need to strip "and" and commas from the arguments before concatenating the string
    first_name = str(row['First_Name'])
    child_name = str(row['Child_Names'])
    return str(str.replace(first_name,'and','') + ' ' + str.replace(child_name,',',''))    

df['All_Names'] = df.apply(concat_names, axis=1)
df.head(5)


In [ ]:
# Not Needed: This cell will attempt to add a column that counts the number of names
def count_names(row):
    # this function will count the number of names in the All_Names column
    return int(len(str(row['All_Names']).split()) )    

df['Name_Count'] = df.apply(count_names, axis=1)  # iterate over the column
df.head(5)

In [ ]:
#Good Code: This cell will attempt to create a new dataframe with rows for each person

phone_cf = df.copy(deep=True)

# Keep only the columns needed to create the contacts (FamilyId will be the key)
phone_cf.drop(columns=['Emails', 'Last_Name', 'First_Name', 'Child_Names'], inplace=True)
phone_cf.drop(columns=['Address', 'City', 'State','ZIP'], inplace=True)
phone_cf.drop(columns=['AnniversaryDate', 'Additional_Details'], inplace=True)
phone_cf.drop(columns=['Notes', 'Active', 'ImageExists', 'DateModified'], inplace=True)
phone_cf.head(10)



In [ ]:
#Good Code: This cell will attempt to create a new dataframe with rows for each person

email_cf = df.copy(deep=True)

# Keep only the columns needed to create the contacts (FamilyId will be the key)
email_cf.drop(columns=['Phone_Numbers', 'Last_Name', 'First_Name', 'Child_Names'], inplace=True)
email_cf.drop(columns=['Address', 'City', 'State','ZIP'], inplace=True)
email_cf.drop(columns=['AnniversaryDate', 'Additional_Details'], inplace=True)
email_cf.drop(columns=['Notes', 'Active', 'ImageExists', 'DateModified'], inplace=True)
email_cf.head(10)

In [ ]:
#Not Needed: Experiment: This cell will attempt to create rows for each owner


def create_owners(row):
    # For each row, parse the phone numbers and add it to the dataframe
    phone_obj = pandas.DataFrame(columns=['Owner', 'Phone_Number'])
    phone_list = [x.strip() for x in row['Phone_Numbers'].split(',')]
    for i in phone_list:
        phone_s = ''.join(map(str, i)) # convert list to string
        chars = '()'  # Identify the characters to be removed
        res = phone_s.translate(str.maketrans('','', chars)) #Remove parens
        phone_pair = [x.strip() for x in res.split(" ")]
        # print(phone_pair, type(phone_pair))
        phone_obj = phone_obj.append({'Owner': phone_pair[2], 'Phone_Number': phone_pair[1]}, ignore_index=True)
    return True    

cf['Owner'] = cf.apply(create_owners, axis=1)  # iterate over the column
cf.head(5)
phone_obj.head(5)

In [ ]:
# Not Needed: Experiment: This cell will attempt to Parse Phone Numbers
phone1 = ['515-230-2171 (Caleb), 615-310-7347 (Katy)']   # type=list
phone_str = ' '.join(map(str, phone1))  # convert to type=string
phone_list = [x.strip() for x in phone_str.split(',')]
print(type(phone_list), phone_list, len(phone_list))
for i in phone_list:
    phone_s = ''.join(map(str, i)) # convert list to string
    chars = '()'  # Identify the characters to be removed
    res = phone_s.translate(str.maketrans('','', chars)) #Remove parens
    print(res, len(res))
    

In [ ]:
#Experiment: This cell will attempt to work with just the Phone Number column

phone_numbers = cf['Phone_Numbers']
# print(type(phone_numbers), phone_numbers)
for ind in cf['Phone_Numbers']:
    phone_s = ''.join(map(str, ind)) # convert list to string
    chars = '()'  # Identify the characters to be removed
    res = phone_s.translate(str.maketrans('','', chars)) #Remove parens
    phone_pair = [x.strip() for x in res.split(" ")]
    print(phone_pair, type(phone_pair))
    phone_obj = phone_obj.append({'Owner': phone_pair[2], 'Phone_Number': phone_pair[1]}, ignore_index=True)
phone_obj.head()

In [ ]:
# Good Code:  This cell will attempt to create multiple rows for each phone
phone_cf['Phone_Numbers'] = phone_cf['Phone_Numbers'].str.split(',')
phone_cf =  phone_cf.explode('Phone_Numbers').reset_index(drop=True)
cols = list(phone_cf.columns)
cols.append(cols.pop(cols.index('FamilyId')))
phone_cf = phone_cf[cols]
phone_cf.head(10)

In [ ]:
# Good Code:  This cell will attempt to create multiple rows for each email
email_cf['Emails'] = email_cf['Emails'].str.split(',')
email_cf =  email_cf.explode('Emails').reset_index(drop=True)
cols = list(email_cf.columns)
cols.append(cols.pop(cols.index('FamilyId')))
email_cf = email_cf[cols]
email_cf.head(10)

Next steps:
    Need to separate the dataframes into 2 dataframes (phone and mail)
    Each dataframe should only contain the FamilyId and the other axis
    Then need to strip the owner from each dataframe and put it into a new column

In [ ]:
#Doesn't work - Experiment: This cell will attempt to populate the phone owner


def create_phone_owners(row):
    # For each row, parse the phone numbers and add it to the dataframe
        phone_list = row['Phone_Numbers']  
        # print('phone_list= ', phone_list, type(phone_list))
        phone_map = ''.join(map(str, phone_list)) # convert list to string
        chars = '()'  # Identify the characters to be removed
        no_parens = phone_map.translate(str.maketrans('','', chars)) #Remove parens

        #  print('no_parens= ', no_parens, type(no_parens))
        owner_list = no_parens.split(' ')
        #  print('owner_list= ', owner_list, type(owner_list), len(owner_list))

        if len(owner_list) == 2:
            owner = owner_list[1]
        else:
            owner = ''       
        # print('owner= ', owner, type(owner))
        return str(owner)    

phone_cf['Owner'] = phone_cf.apply(create_phone_owners, axis=1)
# phone_cf['Owner'] = phone_cf.apply(create_phone_owners).reset_index(drop=True)
phone_cf.head(10)

In [ ]:
# Code Snippet.... may use later
# https://stackoverflow.com/questions/46585193/pandas-error-in-python-columns-must-be-same-length-as-key
email_cf = email_cf['Emails'].str.split(n=1, expand=True)
email_cf.columns = ['email{}'.format(x+1) for x in email_cf.columns]
email_cf.head(10)

In [ ]:
# Good Code and Successful Test: This cell will clean up phone numbers
# Note:  Need to add code to return a blank string if no phone number present
phone_list = phone_cf['Phone_Numbers'].tolist()
for row in phone_list:
    phone_row = ''.join(map(str, row)) # convert list to string
    for match in phonenumbers.PhoneNumberMatcher(phone_row, "US"):
        print(phonenumbers.format_number(match.number,                              phonenumbers.PhoneNumberFormat.NATIONAL))


In [ ]:
# Good Code: Combining a function with phone number cleanup
# Note:  Need to add code to return a blank string if no phone number present
def find_phone_number(row):
    phone_row = ''.join(map(str, row))
    for match in phonenumbers.PhoneNumberMatcher(phone_row, "US"):
        return str(phonenumbers.format_number(match.number,                              phonenumbers.PhoneNumberFormat.NATIONAL))
phone_cf['PhoneG']=phone_cf['Phone_Numbers'].apply(lambda x: find_phone_number(x))
phone_cf.head(10)

In [ ]:
# Good Code: Combining a function with phone owner cleanup
def find_phone_owner(row):
    phone_row = ''.join(map(str, row))
    if phone_row.rfind(')'):
        owner = phone_row.rstrip("*")[phone_row.rfind('(')+1:-1]
    else:
        owner = ''
    return str(owner.strip())
phone_cf['Phone_Owner']=phone_cf['Phone_Numbers'].apply(lambda x: find_phone_owner(x))
phone_cf.head(10)

In [ ]:
# Good Code: Combining a function with email owner cleanup
def find_email_owner(row):
    email_row = ''.join(map(str, row))
    if email_row.rfind(')'):
        owner = email_row.rstrip("*")[email_row.rfind('(')+1:-1]
    else:
        owner = ''
    return str(owner.strip())
email_cf['Email_Owner']=email_cf['Emails'].apply(lambda x: find_email_owner(x))
email_cf.head(10)

In [ ]:
# Good Code: THis cell will attempt to merge the phone_cf dataframe with the main
merge_df = pandas.merge(df, phone_cf)
merge_df.head(5)

In [ ]:
# Experiment: Combining a function with email cleanup
# Note:  Need to add code to return a blank string if no email present
def find_email_address(row):
    email_row = ''.join(map(str, row))
    
    if email_row.rfind(')'):
        email_address = email_row.split(" ")[0]
    else:
        email_address = ''
    return str(email_address)
email_cf['Email_Address']=email_cf['Emails'].apply(lambda x: find_email_address(x))
email_cf.head(10)

In [ ]:
# Good Code: THis cell will attempt to merge the phone_cf dataframe with the main
merge_df = pandas.merge(merge_df, email_cf)
merge_df.head(5)